<a href="https://colab.research.google.com/github/hbechara/HertieML/blob/main/Lesson11UnsupervisedML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Exercise: The classic Olivetti faces dataset contains 400 grayscale 64 × 64–pixel images of faces. Each image is flattened to a 1D vector of size 4,096. 40 different people were photographed (10 times each), and the usual task is to train a model that can predict which person is represented in each picture. Load the dataset using the `sklearn.datasets.fetch_olivetti_faces()` function.*

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

olivetti = fetch_olivetti_faces()

Split it into a training set, a validation set, and a test set (note that the dataset is already scaled between 0 and 1). 

Tip: Use stratified sampling to ensure that there are the same number of images per person in each set.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

strat_split = StratifiedShuffleSplit(n_splits=1, test_size=40, random_state=42)
train_valid_idx, test_idx = next(strat_split.split(olivetti.data, olivetti.target))
X_train_valid = olivetti.data[train_valid_idx]
y_train_valid = olivetti.target[train_valid_idx]
X_test = olivetti.data[test_idx]
y_test = olivetti.target[test_idx]

strat_split = StratifiedShuffleSplit(n_splits=1, test_size=80, random_state=42)
train_idx, valid_idx = next(strat_split.split(X_train_valid, y_train_valid))
X_train = X_train_valid[train_idx]
y_train = y_train_valid[train_idx]
X_valid = X_train_valid[valid_idx]
y_valid = y_train_valid[valid_idx]

Reduce the data's dimensionality using PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.99)
oli_reduced_data = pca.fit_transform(olivetti.data)

pca.n_components_

260

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=40, random_state=42)


for train_valid_index, test_index in sss.split(olivetti.data, olivetti.target):
  print("TRAIN_VALID:", train_valid_index, "TEST:", test_index)
  oli_train_valid = oli_reduced_data[train_valid_index]
  oli_labels_train_valid = olivetti.target[train_valid_index]
  oli_test = oli_reduced_data[test_index]
  oli_labels_test = olivetti.target[test_index]

TRAIN_VALID: [236 330 109 369 251  60  63 288  95 262  65 276 213  86 261 302 337 345
 105 340 216 245 317 167 184 134  77 296 305 274 263  94 351  14 121 311
  74 112 116  88 158  11 358 282 380 283  38  70 204 395 290 250 334 399
 336 287   5 232 144  40  82 202 145 346 364 187  87 267 222 131 227 326
  64 398  32  72 175 237  42  79 106 275 367 143  45 393 207 316  71  21
  53 177 344  48 165 215   9 252 224 235 189 156 374 180 342  93 196 313
 249  13 383 181  47 329 123 256  62 234 163 327 151 259 130 142  23 388
 335 239 293 124  92  59 223  31   3 272 244  29 266 125 229  61 324 394
  36 200 225 242 349 128 386 389 365 303 231 348 381 226 281 233  20 301
 172  84 366 298 149 117 129 170 199 271 273   4  99  81  34 148 396 387
 280 155 268  43  46  80 185 310  78  73  18 378 186 279 286 373 240 300
 188 193 157 162  97 174 371   1  85  25  17   0 321 294 291 339 308 153
 322  33 253 260  10  28 323 241  30 111 353  90 357 372 246 356 228 214
 355 103 314 377 397 221  37 208  96 3

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=80, random_state=42)
for train_index, valid_index in sss.split(oli_train_valid, oli_labels_train_valid):
  print("TRAIN:", train_index, "VALID:", test_index)
  oli_train = oli_train_valid[train_index]
  oli_labels_train = oli_labels_train_valid[train_index]
  oli_valid = oli_train_valid[valid_index]
  oli_labels_valid = oli_labels_train_valid[valid_index]

TRAIN: [127  80  24  10 163  12 182 183  37 293 131  53   0 215 358 277 102  77
 177 159  18 275 144 330  44 305 272 166  17 250  41 271  90 160  52  79
 190 171 205 269  58 225 255  81  57 120 143 139 117   2 104 112  98  95
 191 141 320 321 281 238 290 209 352 148  50 211 325 167  35  82 180 208
 310  51  96 334 179 229 256  91 198 257 129 213 165 245 220 106 348 194
 114 216   7 199 339 200  40  27 279 254 138 346 284  78   3  45 154 351
 193  43 235  32  63 125 242 219 105 239 206 137 103 329 232 308 168 299
 274 246 278 317 285 169 294  65 153 240 185 234 327 230   4 309 151 203
 145 306 249 197 276   9  36  47  55  72 261 126 181 231 336 353 101  15
 189  30 324 107 314 130 268 121  64 187  20 162 175 340 237 266 161 227
 297 260 241 201  48  61  33   8 221 136 354  60  85 247 335 283 233 252
  16 311 333 359 328   5 244 142 295 152 186  70 316  29 298 149 196 350
  94 289  21 123 210  13  26  87 133 251 344 174 178  28 146 322 172 301
  71  14 226 265 115  42  92  69 224 259  54

 Cluster the images using K-Means, and ensure that you have a good number of clusters.

In [ ]:
from sklearn.cluster import KMeans

k_range = range(5, 150, 5)
kmeans_per_k = []
for k in k_range:
    print("k={}".format(k))
    kmeans = KMeans(n_clusters=k, random_state=42).fit(oli_train)
    kmeans_per_k.append(kmeans)

In [ ]:
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib.pyplot as plt

silhouette_scores = [silhouette_score(oli_train, model.labels_) for model in kmeans_per_k]
best_index = np.argmax(silhouette_scores)
best_k = k_range[best_index]
print("Best k = {}".format(best_k))

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(k_range, silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.plot(best_k, best_score, "rs")
plt.show()

What is the best k for this dataset?

As a baseline, train a classifier to predict which person is represented in each picture, and evaluate it on the validation set.

To keep Slava happy, we will make it a Random Forest Classifier

In [ ]:
# Thilos solution
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf_RF = RandomForestClassifier(random_state=42)
clf_RF.fit(oli_train, oli_labels_train)
y_valid_predict = clf_RF.predict(oli_valid)
accuracy_score(oli_labels_valid, y_valid_predict)

In [ ]:
for k in range(len(k_range)):
  if k_range[k] == 105:
    print(k)
    index_105 = k

In [ ]:
best_model = kmeans_per_k[index_105]
oli_train_best = best_model.transform(oli_train)
oli_valid_best = best_model.transform(oli_valid)
oli_test_best = best_model.transform(oli_test)

clf_RF2 = RandomForestClassifier(random_state=42)
clf_RF2.fit(oli_train_best, oli_labels_train)
y_valid_predict = clf_RF2.predict(oli_valid_best)
accuracy_score(oli_labels_valid, y_valid_predict)

Let's use K-Means as a dimensionality reduction tool, and train a classifier on the reduced set.

Do this for an index of 100 first. Then try a few more to see if you can get a higher score.

Finally, append the features from the reduced set to the original features and see if you can beat the baseline.